# 3. Dynamic Loading

## Normal Importing

In [2]:
import json

In [3]:
json.__all__

['dump',
 'dumps',
 'load',
 'loads',
 'JSONDecoder',
 'JSONDecodeError',
 'JSONEncoder']

In [4]:
json.__doc__

'JSON (JavaScript Object Notation) <https://json.org> is a subset of\nJavaScript syntax (ECMA-262 3rd edition) used as a lightweight data\ninterchange format.\n\n:mod:`json` exposes an API familiar to users of the standard library\n:mod:`marshal` and :mod:`pickle` modules.  It is derived from a\nversion of the externally maintained simplejson library.\n\nEncoding basic Python object hierarchies::\n\n    >>> import json\n    >>> json.dumps([\'foo\', {\'bar\': (\'baz\', None, 1.0, 2)}])\n    \'["foo", {"bar": ["baz", null, 1.0, 2]}]\'\n    >>> print(json.dumps("\\"foo\\bar"))\n    "\\"foo\\bar"\n    >>> print(json.dumps(\'\\u1234\'))\n    "\\u1234"\n    >>> print(json.dumps(\'\\\\\'))\n    "\\\\"\n    >>> print(json.dumps({"c": 0, "b": 0, "a": 0}, sort_keys=True))\n    {"a": 0, "b": 0, "c": 0}\n    >>> from io import StringIO\n    >>> io = StringIO()\n    >>> json.dump([\'streaming API\'], io)\n    >>> io.getvalue()\n    \'["streaming API"]\'\n\nCompact encoding::\n\n    >>> import json\

In [5]:
json.__file__

'/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/json/__init__.py'

In [2]:
import sys

# List all modules that have been imported
print(list(sys.modules.keys()))

['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_distutils_hack', 'site', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib._abc', 'types', 'importlib.util', 'runpy', '_operator', 'operator', 'itertools', 'keyword', 'reprlib', '_collections', 'collections', '_functools', 'functools', 'enum', '_sre', 're._constants', 're._parser', 're._casefix', 're._compiler', 'copyreg', 're', 'collections.abc', 'contextlib', '_typing', 'typing.io', 'typing.re', 'typing', 'ipykernel._version', '__future__', '_json', 'json.scanner', 'json.decoder', 'json.encoder', 'json', 'errno', '_locale', 'locale', 'signal', 

In [7]:
!python3 -c "import sys; print(list(sys.modules.keys()))"

['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_distutils_hack', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib._abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', '_functools', 'functools', 'contextlib', 'importlib.util', 'importlib.machinery', 'google', 'google.logging', 'mpl_toolkits', 'enum', '_sre', 're._constants', 're._parser', 're._casefix', 're._compiler', 'copyreg', 're', 'sitecustomize', 'site']


## Dynamic importing

In [8]:
import importlib

math = importlib.import_module("math")
math.sqrt(4)

2.0

In [10]:
from collections.abc import Iterator
from types import ModuleType
import pkgutil

def import_submodules(package_name: str) -> Iterator[ModuleType]:
    package = importlib.import_module(package_name)

    for _, name, is_pkg in pkgutil.walk_packages(package.__path__):
        full_name = f"{package_name}.{name}"

        module = importlib.import_module(full_name)
        yield module

        if is_pkg:
            yield from import_submodules(full_name)

In [23]:
try:
    for module in import_submodules("pydantic"):
        print(module)
except:
    pass

<module 'pydantic._internal' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydantic/_internal/__init__.py'>
<module 'pydantic._internal._config' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_config.py'>
<module 'pydantic._internal._core_metadata' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_core_metadata.py'>
<module 'pydantic._internal._core_utils' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_core_utils.py'>
<module 'pydantic._internal._dataclasses' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_dataclasses.py'>
<module 'pydantic._internal._decorators' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydantic/_internal/_decorators.py'>
<module 'pydantic._internal._decorators_v1' from '/usr/local/Cellar/python@3.12/3.12.0/lib/python3.12/site-packages/pydan

In [49]:
from pathlib import Path
import importlib.util

def import_from_path(path: Path):
    spec = importlib.util.spec_from_file_location(path.stem, path)

    if not spec or not spec.loader:
        raise ImportError(f"Cannot load module from path {path}")

    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    return module

In [50]:
some_module = import_from_path(Path("./something.py"))
some_module.greeting("World")

'Hello, World!'